In [126]:
## Import Libraries 
import numpy as np      # for array operations
import pandas as pd     # for reading data operations

from keras.preprocessing.text import Tokenizer          # for tokenizing text
from keras.preprocessing.sequence import pad_sequences  # for padding sentences with zeros. To make the sentence length same
from keras.utils import to_categorical                  # for one-hot encoding of the labels
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from keras.layers import SimpleRNN, LSTM, GRU, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Embedding, GlobalAvgPool1D
from keras.models import Sequential, Model
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.collocations import BigramCollocationFinder 
from wordcloud import WordCloud

import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_colwidth',-1)

import urllib.request as url
from bs4 import BeautifulSoup as bs
import re
import requests
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
random.seed(123)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import os
import random
import string
import datetime as dt

# import warnings
# warnings.filterwarnings('ignore','RuntimeWarning')

import nltk
import re
from nltk.corpus import stopwords
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from wordcloud import WordCloud

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import nltk

from sklearn.metrics import f1_score


In [112]:
import pickle
filename = 'f1.csv'
infile = open(filename,'rb')
train_text_dt = pickle.load(infile)
infile.close()

In [113]:
train_text_dt.shape

(3000, 26)

In [114]:
train_text_dt.columns

Index(['review', 'displayImageUrl', 'isVerified', 'isSuperReviewer',
       'hasSpoilers', 'hasProfanity', 'createDate', 'updateDate', 'score',
       'timeFromCreation', 'user.realm', 'user.displayName', 'sentiment',
       'Create_day', 'create_time', 'update_day', 'update_time',
       'review_update_date', 'review_processed_1', 'review_processed_2',
       'review_processed_3', 'polarity', 'review_len', 'word_count',
       'review_len_cat', 'word_count_cat'],
      dtype='object')

In [115]:
train_text_dt = train_text_dt.drop(labels=['displayImageUrl', 'isVerified', 'isSuperReviewer',
       'hasSpoilers', 'hasProfanity', 'createDate', 'updateDate', 'score',
       'timeFromCreation', 'user.realm', 'user.displayName',
       'Create_day', 'create_time', 'update_day', 'update_time',
       'review_update_date'], axis=1)

In [116]:
train_text_dt.sample(2)

,review,sentiment,review_processed_1,review_processed_2,review_processed_3,polarity,review_len,word_count,review_len_cat,word_count_cat
559,The movie missed a lot of lines from the cartoon version,0,"[movie, missed, line, cartoon, version]","[movie, missed, line, cartoon, version]",The movie missed a lot of lines from the cartoon version,0.0,56,11,lessthan150,10-30
1888,The Lion King was a great family movie absolutely loved it!!!,1,"[lion, king, great, family, movie, absolutely, loved, it!!!]","[Lion, King, great, family, movie, absolutely, loved]",The Lion King was a great family movie absolutely loved it!!!,0.9,61,11,lessthan150,10-30


In [117]:
train_text_dt['review_processed_2'] = train_text_dt["review_processed_2"].apply(lambda x:' '.join(x))


In [118]:
train_text_dt['review_processed_1'] = train_text_dt['review_processed_1'].apply(lambda x:' '.join(x))

In [119]:
train_text_dt['sentiment'] = train_text_dt['sentiment'].replace({0:1, 1:0})


### Review_processed_1

#### count vec

In [121]:
from sklearn.model_selection import train_test_split
X_train_r1, X_test_r1, y_train_r1, y_test_r1 = train_test_split(train_text_dt['review_processed_1'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [123]:
print(len(X_train_r1))
print(len(X_test_r1))

2250
750


In [124]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r1 = cv.fit_transform(X_train_r1)
tdm_test_r1 = cv.transform(X_test_r1)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r1 = tfidf_vectorizer.fit_transform(X_train_r1)
tfidf_test_r1 = tfidf_vectorizer.transform(X_test_r1)


In [127]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tdm_train_r1,y_train_r1) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r1 = lr_clf.predict(tdm_train_r1)

#predicting on test data
test_pred_r1 = lr_clf.predict(tdm_test_r1)

In [128]:
print("train cfm :","\n", confusion_matrix(y_train_r1,train_pred_r1))
print("train cfm :","\n", confusion_matrix(y_test_r1,test_pred_r1))

train cfm : 
 [[1602   24]
 [ 113  511]]
train cfm : 
 [[521  30]
 [ 92 107]]


In [129]:
f1_train_r1_log=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_log=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_log)
print(f1_test_r1_log)

0.8817946505608283
0.636904761904762


#### tfidf

In [154]:
from sklearn.model_selection import train_test_split
X_train_r1, X_test_r1, y_train_r1, y_test_r1 = train_test_split(train_text_dt['review_processed_1'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [155]:
print(len(X_train_r1))
print(len(X_test_r1))

2250
750


In [156]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r1 = cv.fit_transform(X_train_r1)
tdm_test_r1 = cv.transform(X_test_r1)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r1 = tfidf_vectorizer.fit_transform(X_train_r1)
tfidf_test_r1 = tfidf_vectorizer.transform(X_test_r1)


In [157]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tfidf_train_r1,y_train_r1) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r1 = lr_clf.predict(tfidf_train_r1)

#predicting on test data
test_pred_r1 = lr_clf.predict(tfidf_test_r1)

In [158]:
print("train cfm :","\n", confusion_matrix(y_train_r1,train_pred_r1))
print("train cfm :","\n", confusion_matrix(y_test_r1,test_pred_r1))

train cfm : 
 [[1621    5]
 [ 277  347]]
train cfm : 
 [[540  11]
 [132  67]]


In [159]:
f1_train_r1_log=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_log=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_log)
print(f1_test_r1_log)

0.7110655737704918
0.4837545126353791


#### random forest

In [178]:
#build a random forest classifiers
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()


In [179]:
rf_clf = rf.fit(tdm_train_r1,y_train_r1)
#prediction on train data
train_pred_r1 = rf_clf.predict(tdm_train_r1)

#predicting on test data
test_predr1 = rf_clf.predict(tdm_test_r1)

In [185]:
f1_train_r1_rf=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_rf=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_rf)
print(f1_test_r1_rf)

0.955
0.4837545126353791


#### svm/cv

In [207]:
## Build a SVM Classifier
from sklearn.svm import SVC

## Create an SVC object and print it to see the default arguments
svc = SVC()
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [208]:
svc_c10_rbf = SVC(C=10,kernel='rbf')
svc_c10_rbf

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [209]:
## Fit the model svc_c10_rbf on the train data (X_train,y_train)
svc_c10_rbf.fit(X = tdm_train_r1,y = y_train_r1)

#prediction on train data
train_pred_r1 = svc_c10_rbf.predict(tdm_train_r1)

#predicting on test data
test_pred_r1 = svc_c10_rbf.predict(tdm_test_r1)

In [210]:
f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_svc)
print(f1_test_r1_svc)

0.18497109826589592
0.13761467889908255


#### Grid search on svc

In [212]:
## Use Grid Search for parameter tuning

from sklearn.model_selection import GridSearchCV

svc_grid = SVC()
 

param_grid = {

'C': [0.001, 0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1], 
'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

 
svc_cv_grid = GridSearchCV(estimator = svc_grid, param_grid = param_grid, cv = 10)

In [213]:
## Fit the grid search model
svc_cv_grid.fit(X = tdm_train_r1, y = y_train_r1)

train_pred_r1 = svc_cv_grid.predict(tdm_train_r1)

test_pred_r1 = svc_cv_grid.predict(tdm_test_r1)

In [214]:
## Print best score and parameters
print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8093333333333333 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [215]:
svc_clf_grid = SVC(C=10, gamma=0.01, kernel= 'rbf')

In [217]:
f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_svc)
print(f1_test_r1_svc)

0.8653500897666068
0.6257668711656441


In [218]:
## Fit the grid search model
svc_cv_grid.fit(X = tfidf_train_r1, y = y_train_r1)

train_pred_r1_tf = svc_cv_grid.predict(tfidf_train_r1)

test_pred_r1_tf = svc_cv_grid.predict(tfidf_test_r1)

In [219]:
f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_svc)
print(f1_test_r1_svc)

0.8653500897666068
0.6257668711656441


In [220]:
## Print best score and parameters
print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8257777777777778 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [221]:
svc_clf_grid = SVC(C=10, gamma=0.1, kernel= 'rbf')

In [222]:
f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_svc)
print(f1_test_r1_svc)

0.8653500897666068
0.6257668711656441


### review processed 2

In [130]:
from sklearn.model_selection import train_test_split
X_train_r2, X_test_r2, y_train_r2, y_test_r2 = train_test_split(train_text_dt['review_processed_2'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [131]:
print(len(X_train_r2))
print(len(X_test_r2))

2250
750


In [132]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r2 = cv.fit_transform(X_train_r2)
tdm_test_r2 = cv.transform(X_test_r2)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r2 = tfidf_vectorizer.fit_transform(X_train_r2)
tfidf_test_r2 = tfidf_vectorizer.transform(X_test_r2)


In [133]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tdm_train_r2,y_train_r2) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r2 = lr_clf.predict(tdm_train_r2)

#predicting on test data
test_pred_r2 = lr_clf.predict(tdm_test_r2)

In [134]:
print("train cfm :","\n", confusion_matrix(y_train_r2,train_pred_r2))
print("train cfm :","\n", confusion_matrix(y_test_r2,test_pred_r2))

train cfm : 
 [[1599   27]
 [ 123  501]]
train cfm : 
 [[524  27]
 [ 93 106]]


In [135]:
f1_train_r2_log=f1_score( y_train_r2, train_pred_r2, labels=None, pos_label=1, average='binary')
f1_test_r2_log=f1_score( y_test_r2, test_pred_r2, labels=None, pos_label=1, average='binary')
print(f1_train_r2_log)
print(f1_test_r2_log)

0.8697916666666667
0.6385542168674698


#### tfidf

In [160]:
from sklearn.model_selection import train_test_split
X_train_r2, X_test_r2, y_train_r2, y_test_r2 = train_test_split(train_text_dt['review_processed_2'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [161]:
print(len(X_train_r2))
print(len(X_test_r2))

2250
750


In [162]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r2 = cv.fit_transform(X_train_r2)
tdm_test_r2 = cv.transform(X_test_r2)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r2 = tfidf_vectorizer.fit_transform(X_train_r2)
tfidf_test_r2 = tfidf_vectorizer.transform(X_test_r2)


In [163]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tfidf_train_r2,y_train_r2) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r2 = lr_clf.predict(tfidf_train_r2)

#predicting on test data
test_pred_r2 = lr_clf.predict(tfidf_test_r2)

In [164]:
print("train cfm :","\n", confusion_matrix(y_train_r2,train_pred_r2))
print("train cfm :","\n", confusion_matrix(y_test_r2,test_pred_r2))

train cfm : 
 [[1620    6]
 [ 273  351]]
train cfm : 
 [[540  11]
 [132  67]]


In [165]:
f1_train_r2_log=f1_score( y_train_r2, train_pred_r2, labels=None, pos_label=1, average='binary')
f1_test_r2_log=f1_score( y_test_r2, test_pred_r2, labels=None, pos_label=1, average='binary')
print(f1_train_r2_log)
print(f1_test_r2_log)

0.7155963302752292
0.4837545126353791


#### random forest

In [181]:
#build a random forest classifiers
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()


In [182]:
rf_clf = rf.fit(tdm_train_r2,y_train_r2)
#prediction on train data
train_pred_r2 = rf_clf.predict(tdm_train_r2)

#predicting on test data
test_predr2 = rf_clf.predict(tdm_test_r2)

In [184]:
f1_train_r2_rf=f1_score( y_train_r2, train_pred_r2, labels=None, pos_label=1, average='binary')
f1_test_r2_rf=f1_score( y_test_r2, test_pred_r2, labels=None, pos_label=1, average='binary')
print(f1_train_r2_rf)
print(f1_test_r2_rf)

0.9429530201342282
0.4837545126353791


#### svm

In [203]:
## Build a SVM Classifier
from sklearn.svm import SVC

## Create an SVC object and print it to see the default arguments
svc = SVC()
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [204]:
svc_c10_rbf = SVC(C=10,kernel='rbf')
svc_c10_rbf

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [205]:
## Fit the model svc_c10_rbf on the train data (X_train,y_train)
svc_c10_rbf.fit(X = tdm_train_r2,y = y_train_r2)

#prediction on train data
train_pred_r2 = svc_c10_rbf.predict(tdm_train_r2)

#predicting on test data
test_pred_r2 = svc_c10_rbf.predict(tdm_test_r2)

In [206]:
f1_train_r2_svc=f1_score( y_train_r2, train_pred_r2, labels=None, pos_label=1, average='binary')
f1_test_r2_svc=f1_score( y_test_r2, test_pred_r2, labels=None, pos_label=1, average='binary')
print(f1_train_r2_svc)
print(f1_test_r2_svc)

0.19714285714285712
0.13761467889908255


#### Grid search on svc

In [223]:
## Use Grid Search for parameter tuning

from sklearn.model_selection import GridSearchCV

svc_grid = SVC()
 

param_grid = {

'C': [0.001, 0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1], 
'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

 
svc_cv_grid = GridSearchCV(estimator = svc_grid, param_grid = param_grid, cv = 10)

In [224]:
## Fit the grid search model
svc_cv_grid.fit(X = tdm_train_r1, y = y_train_r1)

train_pred_r1 = svc_cv_grid.predict(tdm_train_r1)

test_pred_r1 = svc_cv_grid.predict(tdm_test_r1)

In [214]:
## Print best score and parameters
print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8093333333333333 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [229]:
svc_clf_grid = SVC(C=10, gamma=0.01, kernel= 'rbf')

In [230]:
## Fit the grid search model
svc_clf_grid.fit(X = tdm_train_r2, y = y_train_r2)

train_pred_r2 = svc_clf_grid.predict(tdm_train_r2)

test_pred_r2 = svc_clf_grid.predict(tdm_test_r2)

In [231]:
f1_train_r2_svc=f1_score( y_train_r2, train_pred_r2, labels=None, pos_label=1, average='binary')
f1_test_r2_svc=f1_score( y_test_r2, test_pred_r2, labels=None, pos_label=1, average='binary')
print(f1_train_r2_svc)
print(f1_test_r2_svc)

0.8514492753623188
0.608695652173913


In [218]:
# ## Fit the grid search model
# svc_cv_grid.fit(X = tfidf_train_r1, y = y_train_r1)

# train_pred_r1_tf = svc_cv_grid.predict(tfidf_train_r1)

# test_pred_r1_tf = svc_cv_grid.predict(tfidf_test_r1)

In [219]:
f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
print(f1_train_r1_svc)
print(f1_test_r1_svc)

0.8653500897666068
0.6257668711656441


In [220]:
# ## Print best score and parameters
# print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8257777777777778 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [233]:
svc_clf_grid = SVC(C=10, gamma=0.1, kernel= 'rbf')

In [234]:
## Fit the grid search model
svc_clf_grid.fit(X = tfidf_train_r2, y = y_train_r2)

train_pred_r2_svc = svc_clf_grid.predict(tfidf_train_r2)

test_pred_r2_svc = svc_clf_grid.predict(tfidf_test_r2)

In [236]:
f1_train_r2_svc=f1_score( y_train_r2, train_pred_r2_svc, labels=None, pos_label=1, average='binary')
f1_test_r2_svc=f1_score( y_test_r2, test_pred_r2_svc, labels=None, pos_label=1, average='binary')
print(f1_train_r2_svc)
print(f1_test_r2_svc)

0.9952
0.6904109589041095


### review processed 3

In [136]:
from sklearn.model_selection import train_test_split
X_train_r3, X_test_r3, y_train_r3, y_test_r3 = train_test_split(train_text_dt['review_processed_3'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [137]:
print(len(X_train_r3))
print(len(X_test_r3))

2250
750


In [138]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r3 = cv.fit_transform(X_train_r3)
tdm_test_r3 = cv.transform(X_test_r3)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r3 = tfidf_vectorizer.fit_transform(X_train_r3)
tfidf_test_r3 = tfidf_vectorizer.transform(X_test_r3)


In [141]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tdm_train_r3,y_train_r3) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r3 = lr_clf.predict(tdm_train_r3)

#predicting on test data
test_pred_r3 = lr_clf.predict(tdm_test_r3)

In [142]:
print("train cfm :","\n", confusion_matrix(y_train_r3,train_pred_r3))
print("train cfm :","\n", confusion_matrix(y_test_r3,test_pred_r3))

train cfm : 
 [[1610   16]
 [ 105  519]]
train cfm : 
 [[511  40]
 [ 87 112]]


In [153]:
f1_train_r3_log=f1_score( y_train_r3, train_pred_r3, labels=None, pos_label=1, average='binary')
f1_test_r3_log=f1_score( y_test_r3, test_pred_r3, labels=None, pos_label=1, average='binary')
print(f1_train_r3_log)
print(f1_test_r3_log)

0.8955996548748921
0.6381766381766382


#### tfidf

In [166]:
from sklearn.model_selection import train_test_split
X_train_r3, X_test_r3, y_train_r3, y_test_r3 = train_test_split(train_text_dt['review_processed_3'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [167]:
print(len(X_train_r3))
print(len(X_test_r3))

2250
750


In [168]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r3 = cv.fit_transform(X_train_r3)
tdm_test_r3 = cv.transform(X_test_r3)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r3 = tfidf_vectorizer.fit_transform(X_train_r3)
tfidf_test_r3 = tfidf_vectorizer.transform(X_test_r3)


In [169]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tfidf_train_r3,y_train_r3) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r3 = lr_clf.predict(tfidf_train_r3)

#predicting on test data
test_pred_r3 = lr_clf.predict(tfidf_test_r3)

In [170]:
print("train cfm :","\n", confusion_matrix(y_train_r3,train_pred_r3))
print("train cfm :","\n", confusion_matrix(y_test_r3,test_pred_r3))

train cfm : 
 [[1622    4]
 [ 259  365]]
train cfm : 
 [[537  14]
 [132  67]]


In [171]:
f1_train_r3_log=f1_score( y_train_r3, train_pred_r3, labels=None, pos_label=1, average='binary')
f1_test_r3_log=f1_score( y_test_r3, test_pred_r3, labels=None, pos_label=1, average='binary')
print(f1_train_r3_log)
print(f1_test_r3_log)

0.7351460221550856
0.47857142857142865


#### random forest

In [186]:
#build a random forest classifiers
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()


In [187]:
rf_clf = rf.fit(tdm_train_r3,y_train_r3)
#prediction on train data
train_pred_r3 = rf_clf.predict(tdm_train_r3)

#predicting on test data
test_pred_r3 = rf_clf.predict(tdm_test_r3)

In [188]:
f1_train_r3_rf=f1_score( y_train_r3, train_pred_r3, labels=None, pos_label=1, average='binary')
f1_test_r3_rf=f1_score( y_test_r3, test_pred_r3, labels=None, pos_label=1, average='binary')
print(f1_train_r3_rf)
print(f1_test_r3_rf)

0.9713349713349713
0.47741935483870973


#### svm

In [199]:
## Build a SVM Classifier
from sklearn.svm import SVC

## Create an SVC object and print it to see the default arguments
svc = SVC()
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [200]:
svc_c10_rbf = SVC(C=10,kernel='rbf')
svc_c10_rbf

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [201]:
## Fit the model svc_c10_rbf on the train data (X_train,y_train)
svc_c10_rbf.fit(X = tdm_train_r3,y = y_train_r3)

#prediction on train data
train_pred_r3 = svc_c10_rbf.predict(tdm_train_r3)

#predicting on test data
test_pred_r3 = svc_c10_rbf.predict(tdm_test_r3)

In [202]:
f1_train_r3_svc=f1_score( y_train_r3, train_pred_r3, labels=None, pos_label=1, average='binary')
f1_test_r3_svc=f1_score( y_test_r3, test_pred_r3, labels=None, pos_label=1, average='binary')
print(f1_train_r3_svc)
print(f1_test_r3_svc)

0.16593886462882096
0.12962962962962962


#### Grid search on svc

In [238]:
## Use Grid Search for parameter tuning

from sklearn.model_selection import GridSearchCV

svc_grid = SVC()
 

param_grid = {

'C': [0.001, 0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1], 
'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

 
svc_cv_grid = GridSearchCV(estimator = svc_grid, param_grid = param_grid, cv = 10)

In [224]:
# ## Fit the grid search model
# svc_cv_grid.fit(X = tdm_train_r3, y = y_train_r3)

# train_pred_r3 = svc_cv_grid.predict(tdm_train_r3)

# test_pred_r3 = svc_cv_grid.predict(tdm_test_r3)

In [214]:
# ## Print best score and parameters
# print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8093333333333333 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [239]:
svc_clf_grid = SVC(C=10, gamma=0.01, kernel= 'rbf')

In [240]:
## Fit the grid search model
svc_clf_grid.fit(X = tdm_train_r3, y = y_train_r3)

train_pred_r3 = svc_clf_grid.predict(tdm_train_r3)

test_pred_r3 = svc_clf_grid.predict(tdm_test_r3)

In [241]:
f1_train_r3_svc=f1_score( y_train_r3, train_pred_r3, labels=None, pos_label=1, average='binary')
f1_test_r3_svc=f1_score( y_test_r3, test_pred_r3, labels=None, pos_label=1, average='binary')
print(f1_train_r3_svc)
print(f1_test_r3_svc)

0.875886524822695
0.6312684365781711


In [218]:
# ## Fit the grid search model
# svc_cv_grid.fit(X = tfidf_train_r1, y = y_train_r1)

# train_pred_r1_tf = svc_cv_grid.predict(tfidf_train_r1)

# test_pred_r1_tf = svc_cv_grid.predict(tfidf_test_r1)

In [219]:
# f1_train_r1_svc=f1_score( y_train_r1, train_pred_r1, labels=None, pos_label=1, average='binary')
# f1_test_r1_svc=f1_score( y_test_r1, test_pred_r1, labels=None, pos_label=1, average='binary')
# print(f1_train_r1_svc)
# print(f1_test_r1_svc)

0.8653500897666068
0.6257668711656441


In [220]:
# ## Print best score and parameters
# print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8257777777777778 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [263]:
svc_clf_grid = SVC(C=8, gamma=0.1, kernel= 'rbf')

In [264]:
## Fit the grid search model
svc_clf_grid.fit(X = tfidf_train_r3, y = y_train_r3)

train_pred_r3_svc = svc_clf_grid.predict(tfidf_train_r3)

test_pred_r3_svc = svc_clf_grid.predict(tfidf_test_r3)

In [265]:
f1_train_r3_svc=f1_score( y_train_r3, train_pred_r3_svc, labels=None, pos_label=1, average='binary')
f1_test_r3_svc=f1_score( y_test_r3, test_pred_r3_svc, labels=None, pos_label=1, average='binary')
print(f1_train_r3_svc)
print(f1_test_r3_svc)

0.9952
0.7379679144385026


### review

In [145]:
from sklearn.model_selection import train_test_split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(train_text_dt['review'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [146]:
print(len(X_train_r))
print(len(X_test_r))

2250
750


In [147]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r = cv.fit_transform(X_train_r)
tdm_test_r = cv.transform(X_test_r)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r = tfidf_vectorizer.fit_transform(X_train_r)
tfidf_test_r = tfidf_vectorizer.transform(X_test_r)


In [149]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tdm_train_r,y_train_r) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r = lr_clf.predict(tdm_train_r)

#predicting on test data
test_pred_r = lr_clf.predict(tdm_test_r)

In [150]:
print("train cfm :","\n", confusion_matrix(y_train_r,train_pred_r))
print("train cfm :","\n", confusion_matrix(y_test_r,test_pred_r))

train cfm : 
 [[1610   16]
 [ 105  519]]
train cfm : 
 [[511  40]
 [ 87 112]]


In [152]:
f1_train_r_log=f1_score( y_train_r, train_pred_r, labels=None, pos_label=1, average='binary')
f1_test_r_log=f1_score( y_test_r, test_pred_r, labels=None, pos_label=1, average='binary')
print(f1_train_r_log)
print(f1_test_r_log)

0.8955996548748921
0.6381766381766382


#### tfidf

In [172]:
from sklearn.model_selection import train_test_split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(train_text_dt['review'],train_text_dt['sentiment'],test_size=0.25,random_state=123)

In [173]:
print(len(X_train_r))
print(len(X_test_r))

2250
750


In [174]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english",strip_accents="unicode",decode_error="ignore")
tdm_train_r = cv.fit_transform(X_train_r)
tdm_test_r = cv.transform(X_test_r)

##########################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_train_r = tfidf_vectorizer.fit_transform(X_train_r)
tfidf_test_r = tfidf_vectorizer.transform(X_test_r)


In [175]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

#training the model
logreg = LogisticRegression()
lr_clf = logreg.fit(tfidf_train_r,y_train_r) #or mat which is in dense format can also be used

#prediction on train data
train_pred_r = lr_clf.predict(tfidf_train_r)

#predicting on test data
test_pred_r = lr_clf.predict(tfidf_test_r)

In [176]:
print("train cfm :","\n", confusion_matrix(y_train_r,train_pred_r))
print("train cfm :","\n", confusion_matrix(y_test_r,test_pred_r))

train cfm : 
 [[1622    4]
 [ 259  365]]
train cfm : 
 [[537  14]
 [132  67]]


In [177]:
f1_train_r_log=f1_score( y_train_r, train_pred_r, labels=None, pos_label=1, average='binary')
f1_test_r_log=f1_score( y_test_r, test_pred_r, labels=None, pos_label=1, average='binary')
print(f1_train_r_log)
print(f1_test_r_log)

0.7351460221550856
0.47857142857142865


#### random forest

In [189]:
#build a random forest classifiers
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()


In [190]:
rf_clf = rf.fit(tdm_train_r,y_train_r)
#prediction on train data
train_pred_r = rf_clf.predict(tdm_train_r)

#predicting on test data
test_pred_r = rf_clf.predict(tdm_test_r)

In [191]:
f1_train_r_rf=f1_score( y_train_r, train_pred_r, labels=None, pos_label=1, average='binary')
f1_test_r_rf=f1_score( y_test_r, test_pred_r, labels=None, pos_label=1, average='binary')
print(f1_train_r_rf)
print(f1_test_r_rf)

0.9617940199335548
0.47297297297297297


#### svm

In [195]:
## Build a SVM Classifier
from sklearn.svm import SVC

## Create an SVC object and print it to see the default arguments
svc = SVC()
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [196]:
svc_c10_rbf = SVC(C=10,kernel='rbf')
svc_c10_rbf

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [197]:
## Fit the model svc_c10_rbf on the train data (X_train,y_train)
svc_c10_rbf.fit(X = tdm_train_r,y = y_train_r)

#prediction on train data
train_pred_r = svc_c10_rbf.predict(tdm_train_r)

#predicting on test data
test_pred_r = svc_c10_rbf.predict(tdm_test_r)

In [198]:
f1_train_r_svc=f1_score( y_train_r, train_pred_r, labels=None, pos_label=1, average='binary')
f1_test_r_svc=f1_score( y_test_r, test_pred_r, labels=None, pos_label=1, average='binary')
print(f1_train_r_svc)
print(f1_test_r_svc)

0.16593886462882096
0.12962962962962962
